**This notebook is covered by the following [license](../License.ipynb)  This note must not be removed**

# Imagetragick Creator

- from https://github.com/mike-williams/imagetragick-poc
- using OpenShift imagestreams and builds https://www.tutorialworks.com/openshift-dockerfile/

In [1]:
crc status

CRC VM:          Running
OpenShift:       Running (v4.7.18)
Disk Usage:      16.36GB of 32.74GB (Inside the CRC VM)
Cache Usage:     13.34GB
Cache Directory: /home/thomas/.crc/cache


In [9]:
crc console --credentials

To login as a regular user, run 'oc login -u developer -p developer https://api.crc.testing:6443'.
To login as an admin, run 'oc login -u kubeadmin -p DL2tm-N8eCB-z7W3U-Df3pI https://api.crc.testing:6443'


In [2]:
oc new-project imagetragick

Now using project "imagetragick" on server "https://api.crc.testing:6443".

You can add applications to this project with the 'new-app' command. For example, try:

    oc new-app rails-postgresql-example

to build a new example application in Ruby. Or use kubectl to deploy a simple Kubernetes application:

    kubectl create deployment hello-node --image=k8s.gcr.io/serve_hostname



In [10]:
oc create imagestream poc

imagestream.image.openshift.io/poc created


In [11]:
oc create -f - <<EOF
apiVersion: build.openshift.io/v1
kind: BuildConfig
metadata:
  name: poc
  labels:
    app: poc
spec:
  source:
    type: Git
    git:
      uri: https://github.com/mike-williams/imagetragick-poc
  strategy:
    type: Docker                      
    dockerStrategy:
      dockerfilePath: Dockerfile    
  output:
    to:
      kind: ImageStreamTag
      name: poc:latest
EOF

buildconfig.build.openshift.io/poc created


In [14]:
oc start-build poc --follow

build.build.openshift.io/poc-1 started
Cloning "https://github.com/mike-williams/imagetragick-poc" ...
	Commit:	832227b757dae0c53746a93a9a734e3943f123be (add readme)
	Author:	Mike Williams <mike.williams@atos.net>
	Date:	Tue May 16 21:27:16 2017 +0100
Caching blobs under "/var/cache/blobs".

Pulling image centos:7 ...
Getting image source signatures
Copying blob sha256:2d473b07cdd5f0912cd6f1a703352c82b512407db6b05b43f2553732b55df3bc
Copying config sha256:eeb6ee3f44bd0b5103bb561b4c16bcb82328cfe5809ab675bb17ab3a16c517c9
Writing manifest to image destination
Storing signatures
STEP 1: FROM centos:7
STEP 2: MAINTAINER 'Mike Williams'
--> c5f18b2bd0f
STEP 3: ENV NODE_ENV Development 
--> 6e7c315ddde
STEP 4: WORKDIR /root
--> a274a183031
STEP 5: RUN yum -y update && yum -y groupinstall "Development tools"
time="2021-09-18T13:45:39Z" level=warning msg="Path \"/run/secrets/etc-pki-entitlement\" from \"/etc/containers/mounts.conf\" doesn't exist, skipping"
time="2021-09-18T13:45:39Z" level=warn

In [16]:
oc get all

NAME              READY   STATUS      RESTARTS   AGE
pod/poc-1-build   0/1     Completed   0          17m

NAME                                 TYPE     FROM   LATEST
buildconfig.build.openshift.io/poc   Docker   Git    1

NAME                             TYPE     FROM          STATUS     STARTED          DURATION
build.build.openshift.io/poc-1   Docker   Git@832227b   Complete   17 minutes ago   15m43s

NAME                                 IMAGE REPOSITORY                                                           TAGS     UPDATED
imagestream.image.openshift.io/poc   default-route-openshift-image-registry.apps-crc.testing/imagetragick/poc   latest   About a minute ago
